In [1]:
import os
import jax
import pickle

os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=12'
jax.config.update('jax_platform_name', 'cpu')

#from jax.config import config; config.update("jax_enable_x64", True)

%load_ext autoreload
%autoreload 2

from brax import envs
from brax.io import html, model
from brax.training import normalization

import flax
import jax.numpy as jnp
from brax.envs import create_fn

from IPython.display import HTML, clear_output

import optax

import matplotlib.pyplot as plt
import numpy as np

from controllers import GruController, MlpController, LinearController

from ce_apg import do_one_rollout, cem_apg

from functools import partial

def visualize(sys, qps, height=480):
  """Renders a 3D visualization of the environment."""
  return HTML(html.render(sys, qps, height=height))

len(jax.devices())

save_dir = "save_ce_apg_ant"

In [ ]:
env_name = "ant" # "acrobot", "inverted_pendulum_swingup", "inverted_double_pendulum_swingup"]:
episode_length = 500
action_repeat = 1
env_fn = create_fn(env_name = env_name, episode_length=episode_length, action_repeat=action_repeat, batch_size=None, auto_reset=False)
env = env_fn()

policy_size = int(2**jnp.ceil(jnp.log2(env.observation_size*4)))
print(policy_size)
policy = GruController(env.observation_size, env.action_size, policy_size)
pickle.dump(policy, open(f"{save_dir}/{env_name}_policy", 'wb'))

seed = 0
inference_fn, params, rewards = cem_apg(env_fn,
                                        20,
                                        key=jax.random.PRNGKey(seed),
                                        episode_length = episode_length,
                                        action_repeat = action_repeat,
                                        apg_epochs = 100,
                                        batch_size = 1,
                                        zero_params=False,
                                        truncation_length = None,
                                        learning_rate = 1e-4,
                                        clipping = 1e9,
                                        initial_std = 0.01,
                                        num_elite = 8,
                                        eps = 0.0,
                                        normalize_observations=True,
                                        policy = policy,
                                        learning_schedule = [-3, -6]
                                       )

model.save_params(f"{save_dir}/{env_name}_params_{seed}", params)
pickle.dump(rewards, open(f"{save_dir}/{env_name}_rewards.pkl{seed}", "wb"))



512
learning rate:  0.0010000000474974513


2022-02-11 13:25:54.449830: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:55] 
********************************
Very slow compile?  If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
Compiling module pmap_do_local_apg.101694
********************************


In [ ]:
from brax.io import model
import ce_apg

jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
#jit_inference_fn = jax.jit(inference_fn)

rollout = []
rng = jax.random.PRNGKey(seed=0)
state = jit_env_reset(rng=rng)
h = np.zeros_like(state.obs)
r = []

while not state.done:
  rollout.append(state)
  r.append(state.reward)
  h, act = jit_inference_fn(params, h, state.obs)
  state = jit_env_step(state, act)

print(sum(r))
HTML(html.render(env.sys, [s.qp for s in rollout]))